# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome()

#passo 1: pegar a cotacao do dolar
#entrar no google
navegador.get('https://www.google.com')

#usando o full xpath (nao recomendado)
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dolar')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = navegador.find_element('xpath', '/html/body/div[7]/div/div[10]/div/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div[3]/div[1]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_dolar)

#passo 2: pegar a cotacao do euro
navegador.get('https://www.google.com')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element('xpath', '/html/body/div[7]/div/div[10]/div/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div[3]/div[1]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)

#passo 3: pegar a cotacao do ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.')

print(cotacao_ouro)

navegador.quit()#fechar o navegador apos o fim de tudo


5.0877
5.516847495
315.12


In [14]:
import pandas as pd


#passo 4: importar a base de dados e atualizar a base de dados
tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [28]:
#passo 5: recalcular os precos
    #atualizar cotacao dolar, euro, ouro.
    

          #linha, coluna

tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

    #recalcular precos
    #preco de comprar = cotacao * preco original
tabela["Preço de Compra"] = tabela["Cotação"] * tabela["Preço Original"]
    #preco de venda= preco de conta * margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]
    
display(tabela)


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda,cotação
0,Câmera Canon,999.99,Dólar,5.087700,5087.649123,1.40,7122.708772,5.0877
1,Carro Renault,4500.00,Euro,5.516847,24825.813728,2.00,49651.627455,NaN
2,Notebook Dell,899.99,Dólar,5.087700,4578.879123,1.70,7784.094509,5.0877
3,IPhone,799.00,Dólar,5.087700,4065.072300,1.70,6910.622910,5.0877
4,Carro Fiat,3000.00,Euro,5.516847,16550.542485,1.90,31446.030722,NaN
5,Celular Xiaomi,480.48,Dólar,5.087700,2444.538096,2.00,4889.076192,5.0877
6,Joia 20g,20.00,Ouro,315.120000,6302.400000,1.15,7247.760000,NaN


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [ ]:
#passo 6: exportar a base atualizada
tabela.to_excel("Produtos Novos.xlsx", index=False) #index=False -> faz com que o index nao seja exportado para o novo arquivo


- Atualizando os preços e o cálculo do Preço Final

### Agora vamos exportar a nova base de preços atualizada

In [ ]:
#instalar 
!pip install selenium